# Decision Analysis - Project 1 #


Mateusz Małecki <br>
Daniel Jankowski 148257

In [1]:
import pandas as pd
import numpy as np
import math
from graphviz import Digraph

In [2]:
data = pd.read_csv("DA_database.csv", decimal=",")

In [3]:
data.head()

,name,prestige,power,price,engine_size,colour_preference,fuel_consumption,mileage,production_year,automatic_gear_box
0,Seat Ibiza 1.2 white,1,90,34900,1.2,2,6.0,90000,2015,0
1,Seat Ibiza 1.4 green,1,85,27800,1.4,4,6.9,82000,2015,0
2,Skoda Fabia grey,4,75,31500,1.0,1,5.8,215000,2019,0
3,Nissan Note grey,3,80,28500,1.2,1,6.0,133000,2014,0
4,MINI Cooper 1.6 blue,7,175,27900,1.6,5,2.0,183000,2006,0


## Decision classes ##

In [99]:
params={
    "prestige":{
    "type":"gain", "func_type":"Type6", "p":2, "q":1, "v":None, "w":5
    },
    "power":{
    "type":"gain", "func_type":"Type5", "p":15, "q":5, "v":None, "w":7
    },
    "price":{
    "type":"cost", "func_type":"Type5", "p":3000, "q":1000, "v":None, "w":8
    },
    "engine_size":{
    "type":"cost", "func_type":"Type5", "p":0.2, "q":0, "v":None, "w":7
    },
    "colour_preference":{
    "type":"gain", "func_type":"Type5", "p":0, "q":0, "v":None, "w":2
    },
    "fuel_consumption":{
    "type":"cost", "func_type":"Type5", "p":0.3, "q":0.1, "v":None, "w":8
    },
    "mileage":{
    "type":"cost", "func_type":"Type6", "p":35000, "q":20000, "v":None, "w":10
    },
    "production_year":{
    "type": "gain", "func_type":"Type5", "p":4, "q":2, "v":None, "w":6
    },
    "automatic_gear_box":{
    "type":"gain", "func_type":"Type1", "p":1, "q":0, "v":None, "w":3
    },
}

## Promethe ##

In [102]:
class Promethee:
    def __init__(self, alternatives: pd.DataFrame, parameters: dict[str, dict]) -> None:
        self.alternatives = alternatives
        self.parameters = parameters
        self.criteria = np.array([x for x in alternatives.columns[1:]])
        self.weights = np.expand_dims(np.array([x['w'] for x in parameters.values()]), axis=(1,2))


    def criteriaComparison(self, criteria: str, a, b, funcType: str) -> float:
        assert(funcType in ["Type1", "Type2", "Type3", "Type4", "Type5", "Type6"])
        p, q = self.parameters[criteria]["p"], self.parameters[criteria]["q"]
        diff = a - b if self.parameters[criteria]["type"] == "gain" else b-a
        if funcType == "Type1":
            return 1 if diff > 0 else 0
        
        elif funcType == "Type2":
            return 1 if diff > q else 0
        
        elif funcType == "Type3":
            if diff > p:
                return 1
            elif diff <= 0:
                return 0
            else:
                return diff / p
        
        elif funcType == "Type4":
            if diff > p:
                return 1
            elif diff < q:
                return 0
            else:
                return 0.5
        
        elif funcType == "Type5":
            if diff > p:
                return 1
            elif diff <= q:
                return 0
            else:
                return (diff - q)/(p-q)
        else:
            return 0 if diff <= 0 else 1 - math.exp(-diff/2) # assuming sd = 1
    
    def computeMarginalPreferencesAllCriteria(self) -> np.array:
        marginalPreferenceIndices = np.zeros((len(self.criteria), len(self.alternatives), len(self.alternatives)))
        for n, criteria in enumerate(self.criteria):
            for i, row in self.alternatives.iterrows():
                for j, row2 in self.alternatives.iterrows():
                    if i == j:
                        continue
                    marginalPreferenceIndices[n][i][j] = self.criteriaComparison(criteria, row[criteria], row2[criteria], self.parameters[criteria]["func_type"])
        return marginalPreferenceIndices
    
    def computeComprehensiveMatrix(self) -> np.array:
        return np.sum(self.computeMarginalPreferencesAllCriteria() * self.weights, axis=0)/sum(self.weights)
    
    def computePositiveFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 1)
    def computeNegativeFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 0)
    
    def prometheeIdrawRanking(self, positive_flows_l, negative_flows_l, incomparable_indifferent, preference):
        ranking = []
        heap = []
        added = []
        for i in range(len(positive_flows_l)):
            a = positive_flows_l.pop(0)[0]
            b = negative_flows_l.pop(0)[0]
            if a in added:
                continue
            if a == b:
                ranking.append([a])
                added.append(a)
            else:
                added.append(a)
                heap.append(a)
                for x in incomparable_indifferent[a]:
                    if a in incomparable_indifferent[x]:
                        heap.append(x)
                        added.append(x)
                ranking.append(heap)
                heap = []
        dot = Digraph()
        for x in self.alternatives["name"]:
            dot.node(x, x)
        for i in range(1, len(ranking)):
                prev_node = ranking[i-1]
                curr_node = ranking[i]
                for x in prev_node:
                    for y in curr_node:
                        if x in preference[y]:
                            dot.edge(y, x)
        dot.render(view=True)

    def prometheeI(self, drawRanking = True, printRelations = False):
        X = self.computeComprehensiveMatrix()
        positive_flows = dict([x for x in zip(self.alternatives["name"], self.computePositiveFlows(X))])
        negative_flows = dict([x for x in zip(self.alternatives["name"], self.computeNegativeFlows(X))])
        positive_flows_l = sorted([x for x in zip(self.alternatives["name"], self.computePositiveFlows(X))], key=lambda x:x[1])
        negative_flows_l = sorted([x for x in zip(self.alternatives["name"], self.computeNegativeFlows(X))], key=lambda x:x[1], reverse=True)

        preferences = {}
        incomparable_indifferent = {}

        for alternative1 in self.alternatives["name"]:
            preferences[alternative1] = []
            incomparable_indifferent[alternative1] = []
            for alternative2 in self.alternatives["name"]:
                if alternative1 == alternative2:
                    continue
                if (positive_flows[alternative1] > positive_flows[alternative2] and negative_flows[alternative1] < negative_flows[alternative2]) or (positive_flows[alternative1] == positive_flows[alternative2] and negative_flows[alternative1] < negative_flows[alternative2]) or (positive_flows[alternative1] > positive_flows[alternative2] and negative_flows[alternative1] == negative_flows[alternative2]):
                    preferences[alternative1].append(alternative2)
                else:
                    incomparable_indifferent[alternative1].append(alternative2)
        if drawRanking:
            self.prometheeIdrawRanking(positive_flows_l, negative_flows_l, incomparable_indifferent, preferences)
        if printRelations:
            for x in self.alternatives["name"]:
                print(f"{x} outranks {preferences[x]}")
                print(f"{x} is incomparable or indifferent to {incomparable_indifferent[x]}\n")
    
    def prometheeIIdrawRanking(self, flow):
        dot = Digraph()
        for x in self.alternatives["name"]:
            dot.node(x, x)
        values = [x[1] for x in flow]
        ranking_temp = [[x[0] for x in flow if x[1] == y] for y in values]
        ranking = []
        for item in ranking_temp:
            if item not in ranking:
                ranking.append(item)
        for i in range(1, len(ranking)):
                prev_node = ranking[i-1]
                curr_node = ranking[i]
                for x in prev_node:
                    for y in curr_node:
                        dot.edge(y, x)
        dot.render(view=True)

    def prometheeIIprintRelations(self, ranking):
        outrank = {x:[] for x in self.alternatives["name"]}
        indifferent = {x:[] for x in self.alternatives["name"]}
        for x in ranking:
            for y in ranking:
                if x == y:
                    continue
                if x[1] > y[1]:
                    outrank[x[0]].append(y[0])
                elif x[1] == y[1]:
                    indifferent[x[0]].append(y[0])
        for x in self.alternatives["name"]:
            print(f"{x} outranks {outrank[x]}")
            print(f"{x} is indifferent to {indifferent[x]}\n")

    def prometheeII(self, drawRanking = True, printRelations = False):
        X = self.computeComprehensiveMatrix()
        balance = sorted([x for x in zip(self.alternatives["name"], self.computePositiveFlows(X) - self.computeNegativeFlows(X))], key=lambda x:x[1])
        if drawRanking: self.prometheeIIdrawRanking(balance)
        if printRelations: self.prometheeIIprintRelations(balance)

        

In [104]:
decisionMaker = Promethee(data, params)
decisionMaker.prometheeII(True, True)

Seat Ibiza 1.2 white outranks ['Mercedes w116 brown', 'Toyota Prius black', 'Abarth Grande Punto white', 'Hyundai i20 white', 'Jaguar xj red', 'MINI Cooper 1.5 grey', 'Volkswagen Polo 1.2 white', 'Skoda Fabia grey', 'Toyota Yaris 1.0 white', 'Nissan Note grey', 'Seat Ibiza 1.4 green', 'Opel Corsa white', 'Alfa Romeo Giuletta white', 'Toyota Yaris 1.3 grey']
Seat Ibiza 1.2 white is indifferent to []

Seat Ibiza 1.4 green outranks ['Mercedes w116 brown', 'Toyota Prius black', 'Abarth Grande Punto white', 'Hyundai i20 white', 'Jaguar xj red', 'MINI Cooper 1.5 grey', 'Volkswagen Polo 1.2 white', 'Skoda Fabia grey', 'Toyota Yaris 1.0 white', 'Nissan Note grey']
Seat Ibiza 1.4 green is indifferent to []

Skoda Fabia grey outranks ['Mercedes w116 brown', 'Toyota Prius black', 'Abarth Grande Punto white', 'Hyundai i20 white', 'Jaguar xj red', 'MINI Cooper 1.5 grey', 'Volkswagen Polo 1.2 white']
Skoda Fabia grey is indifferent to []

Nissan Note grey outranks ['Mercedes w116 brown', 'Toyota Pri

1. What is the domain of the problem about?

    <i>The domain of the problem is set of different cars available on the polish market</i>

2. What is the source of the data?

    <i>The data is taken from the website https://www.otomoto.pl/, where people or companies around Poland are selling their cars.</i>

3. What is the point of view of the decision maker?

    <i>The decision maker is looking for a car suitable for the city, therefore the finally choosen car should be not only powerful, but also economic.</i>

4. What is the number of alternatives considered? Were there more of them in the original data set?

    <i>We've gathered data about 22 cars. As a original dataset we may consider the website https://www.otomoto.pl/. For our search filters there were available more than 3000 cars.</i>

5. Describe one of the alternatives considered (give its name, evaluations, specify preferences for this
alternative)

    <i>Considering the first alternative from our dataset it is a Seat from 2015 year. It has quite small engine (1.2) and fuel consumption (6.0) what is good, because we are looking for the car reasonable for the city (economic). However, the power of the engine is not as good as it could be, it is only 90HP. The price is 34900pln what is the upper boundary looking through the entire dataset. TODO</i>